In [1]:
# Import required librarires and dependencies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [2]:
# Read titanic dataset from seaborn librairy
titanic = sns.load_dataset('titanic')
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


## Create the features matrix with: sex, age, fare, class, embark_town, and alone

In [27]:
X = pd.DataFrame(titanic.drop(['survived','pclass','sibsp','parch','embarked','who','adult_male','deck','alive'],axis=1).copy())
X

,sex,age,fare,class,embark_town,alone
0,male,22.0,7.2500,Third,Southampton,False
1,female,38.0,71.2833,First,Cherbourg,False
2,female,26.0,7.9250,Third,Southampton,True
3,female,35.0,53.1000,First,Southampton,False
4,male,35.0,8.0500,Third,Southampton,True
...,...,...,...,...,...,...
886,male,27.0,13.0000,Second,Southampton,True
887,female,19.0,30.0000,First,Southampton,True
888,female,NaN,23.4500,Third,Southampton,False
889,male,26.0,30.0000,First,Cherbourg,True


## Create target variable 

In [4]:
Y = pd.DataFrame(titanic['alive'].copy())
Y

,alive
0,no
1,yes
2,yes
3,yes
4,no
...,...
886,no
887,yes
888,no
889,yes


## Data Wrangling

### Detect null values

In [28]:
# Merge X and Y for easy manipulation
titanic_df = pd.concat([X,Y],axis=1)
titanic_df

,sex,age,fare,class,embark_town,alone,alive
0,male,22.0,7.2500,Third,Southampton,False,no
1,female,38.0,71.2833,First,Cherbourg,False,yes
2,female,26.0,7.9250,Third,Southampton,True,yes
3,female,35.0,53.1000,First,Southampton,False,yes
4,male,35.0,8.0500,Third,Southampton,True,no
...,...,...,...,...,...,...,...
886,male,27.0,13.0000,Second,Southampton,True,no
887,female,19.0,30.0000,First,Southampton,True,yes
888,female,NaN,23.4500,Third,Southampton,False,no
889,male,26.0,30.0000,First,Cherbourg,True,yes


In [6]:
# Data type
titanic_df.dtypes

sex              object
age             float64
fare            float64
class          category
embark_town      object
alone              bool
alive            object
dtype: object

In [29]:
# Convert alone and class features to object type
titanic_df['alone'] = titanic_df['alone'].astype('object')
titanic_df['class'] = titanic_df['class'].astype('object')

In [30]:
# New Data type
titanic_df.dtypes

sex             object
age            float64
fare           float64
class           object
embark_town     object
alone           object
alive           object
dtype: object

In [9]:
# Null values
titanic_df.isnull().sum()

sex              0
age            177
fare             0
class            0
embark_town      2
alone            0
alive            0
dtype: int64

In [31]:
# Handling null values
mean_age = titanic_df['age'].mean()
mean_age = round(mean_age,5)
print(f'The mean age is:{mean_age}','\n')
print(titanic_df['embark_town'].value_counts())

The mean age is:29.69912 

Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64


In [32]:
# Replace na in age feature by the mean age and na in embark_town by Southampton
titanic_df['age'] = titanic_df['age'].fillna(mean_age)
titanic_df['embark_town'] = titanic_df['embark_town'].fillna('Southampton')

In [33]:
# New check for null values
titanic_df.isnull().sum()

sex            0
age            0
fare           0
class          0
embark_town    0
alone          0
alive          0
dtype: int64

In [36]:
features = ["age", "fare"]
for feature in features:
    min_ = titanic_df[feature].mean() - (3 * titanic_df[feature].std())
    max_ = titanic_df[feature].mean() + (3 * titanic_df[feature].std())
    titanic_df1 = titanic_df[titanic_df[feature] <= max_]
    titanic_df1 = titanic_df[titanic_df[feature] >= min_]
    print(feature,    ":", titanic_df1.shape)

age : (891, 7)
fare : (891, 7)


## Detect and Handling Outliers

In [35]:
# 

In [15]:
# Create new features and target variables
X1 = titanic_df.drop(['sex','class','embark_town', 'alone','alive'],axis=1)
Y1 = titanic_df['alive'] 

In [16]:
X1.shape

(891, 2)

In [23]:
# Remove outliers from numerical features
features = ["age", "fare"]
for feature in features:
    min_ = X1[feature].mean() - (3 * X1[feature].std())
    max_ = X1[feature].mean() + (3 * X1[feature].std())
    X1 = X1[X1[feature] >= min_]
    X1 = X1[X1[feature] <= max_]
    print(feature,    ":", X1.shape)

age : (842, 2)
fare : (829, 2)
